In [ ]:
import pandas as pd 
import numpy as np

albums = pd.read_csv("data/spotify_spotify_albums.csv", index_col=0) 
artists = pd.read_csv("data/spotify_spotify_artists.csv", index_col=0) 
tracks = pd.read_csv("data/spotify_spotify_tracks.csv",index_col=0)

#### Profiling Data

In [ ]:
albums.head() # We see first 5 rows across all columns. This looks to be a wide dataset with many columns containing categorical variables.
# Notice the first column: 'Unnamed: 0'. This is an index column that wasn't read correctly before we re-ran the previous block with 'index_col = 0'. 
# If we preferred not to re-read a large file like this, we could have typed:
  # albums.drop("Unnamed: 0", axis=1, inplace=True)
# Or:
  # albums = albums.iloc[:,1:] 

albums.tail() # Shows last 5 rows and lets us determine number of rows in dataframe (75511)
albums.shape # Gives dimensionality of dataframe

albums.loc[10:20,['name', 'release_date']]

In [ ]:
print(artists.shape)
artists.head() # This has two quantitative variable columns in 'artist_popularity' and 'followers'.
artists.tail()


In [ ]:
print(tracks.shape) # Logically, there are more tracks than albums, and there are more albums than artists
tracks.head() # Another wide dataset with many quantitative variables. Joining with the other two sets could allow for interesting statistical analysis.
tracks.tail()

#### Cleaning and Normalizing Data

In [ ]:
albums.drop_duplicates(inplace=True)
artists.drop_duplicates(inplace=True)
tracks.drop_duplicates(inplace=True)

In [ ]:
def listFill(genre_list): 
  if genre_list == '[]': # didn't set the dtypes on read, so artists.genres was just read as strings
    return np.nan
  else: 
    return genre_list

artists.genres = artists.genres.map(listFill)

In [ ]:
print(tracks.columns)
tracks.lyrics # These lyrics seem to render just fine in LibreOffice Calc. I imagine all the escape characters leading at the front of each line could cause issues for some programs.
tracks.drop('lyrics', axis=1, inplace=True)
print(tracks.columns)

#### Joining Data

##### _Albums and Artists_
I chose a left join on albums because I want to analyze the albums primarily. 
If we did an inner join with artists or a right join on artists, we might lose any albums that didn't have any artists listed with them.
We wouldn't want an outer join because we might include artists with only singles available on spotify and no albums. The rows of those artists would have many empty fields and tell us nothing about the albums.

In [ ]:
albums_artists = pd.merge(albums, artists, how='left', left_on="artist_id", right_on="id")
albums_artists.head(1)

That join did not work. Let's look at the columns where things got messy.

In [ ]:
albums_artists.filter(regex='_[xy]$', axis=1).head(1)

Notice that `albums` and `artists` have multiple columns with the same names.

* `name`: name of the album vs. the name of the artist.
* `id`: id of the album vs. the id of the artist. We tried to join on `albums.artist_id` = `artists.id` but the fact that there is also an `albums.id` column confuses things.
* `type`: 'album' vs. 'artist' (a static, dummy categorical variable we can drop from both dataframes)
* `track_id`: id of every track in each album vs id of one sample song by each artist. We can drop `artists.track_id` when performing the join
* `track_name_prev`: Frankly I'm not sure what this column is. I'm going to drop it in both.

In [ ]:
albums.rename({'name':'album_name','id':'album_id'},axis=1, inplace=True)
artists.rename({'name':'artist_name','id':'artist_id'}, axis=1, inplace=True)
albums.drop(['type', 'track_name_prev'], axis=1, inplace=True)
artists.drop(['type', 'track_name_prev'], axis=1, inplace=True)

In [ ]:
albums_artists2 = pd.merge(albums, artists.drop(['track_id'],axis=1), how='left', on='artist_id')

print(albums_artists2.shape)
albums_artists2.head()

##### _Albums and tracks_

Here I chose a right join on albums to go along with the previous dataframe I just made.

In [ ]:
albums_tracks = pd.merge(tracks, albums, how='right', left_on = ["id","album_id"], right_on = ["track_id", 'album_id'])

As before, it looks like there are some shared column names:

In [ ]:
albums_tracks.filter(regex="_[xy]$").head(1)

In `albums` vs `tracks`, respectively:
* `href`: URL to album vs URL to track
* `uri`: URI for album vs URI for track

In [ ]:
import re
albums_tracks.rename(columns=lambda v: re.sub('_x$','_track',v), inplace = True)
albums_tracks.rename(columns=lambda v: re.sub('_y$','_album',v), inplace = True)

print(albums_tracks.shape)
albums_tracks.head(3)

* `available_markets`: These *should* just be the same, but now I wonder if in certain markets where an album is sold there are tracks that remain unavailable. There could also be some markets where a track is available but not the album it's on; these would have been excluded by our choice of join, however.

In [ ]:
availability_diff = albums_tracks.loc[albums_tracks.available_markets_track != albums_tracks.available_markets_album, ['available_markets_track', 'available_markets_album']]
availability_diff
# Interesting -- 156 such records exist. Hence these columns should indeed be distinct after our join.

#### Analyzing Data

##### _Artists_
1. Which artists appear the most times in `artists`?

In [300]:
artist_appearances = artists.groupby('artist_name').artist_name.count().sort_values(ascending=False)
repeat_artists = artist_appearances[artist_appearances > 1]
repeat_artists
# 551 artists have multiple entries in artists, with Haze and Sasha tied for first.

artist_name
Haze       5
Sasha      5
Luna       4
Alex       4
Plan B     4
          ..
VaVa       2
Chris      2
Belinda    2
Amar       2
Beast      2
Name: artist_name, Length: 551, dtype: int64

In [299]:
# Why are there any artists with multiple entries in the artists dataset? 
# What does it mean for them to have different values in each column?
artists.loc[artists.artist_name.isin(list(repeat_artists.index))].sort_values("artist_name")

,artist_popularity,followers,genres,artist_id,artist_name,track_id
27267,34,125,NaN,2vRMFS86tSrD1pFoY2caFO,1Kilo,2wvtEiHUoRmjrqRfOrY26B
31411,70,2376733,"['brazilian electronica', 'brazilian hip hop',...",6E2st8OqIaS7PU5gj95FSE,1Kilo,5GNT8tXCqPop8K4t3hvVaH
38836,54,144334,"['boy band', 'bubblegum dance', 'dance pop', '...",5lPsVvHVDr6R5mDxRUXdOs,A1,6n2AhEoxaAdArhe6sy6tcC
46459,49,3937,NaN,4QMVGiEbc5NbK2cZzyGgqa,A1,5iemVO9OZkHyGcj8jhqEoM
25539,55,121122,"['brazilian hip hop', 'funk ostentacao']",6Ywb1no2hXLazjIs4LUiQD,ADL,2h608VVCmqu4bIw9ye0FeX
...,...,...,...,...,...,...
41981,50,1103,['lo-fi beats'],5T0wPeBVC0F7uEbIr8JKE5,twuan,6vMNCdqPDr78Pt6FQBuHBS
48141,64,63177,NaN,3abT87tqQ4Q5PA5nw6CYyH,Ángela Aguilar,7L3borCR5Izc7zJjFpjjhh
7345,43,104,NaN,2Qk5eQ0z5H7bk4dyaK6iDo,Ángela Aguilar,2keN15BswHOxejdRAaMT8s
42692,70,1083646,"['latin', 'latin hip hop', 'reggaeton', 'regga...",2OHKEe204spO7G7NcbeO2o,Ñejo,4i7xtBMBUWEecEw780fRtX


2. Which artists have the highest `artist_popularity` rankings?

In [315]:
artists.sort_values('artist_popularity', ascending=False).iloc[0:10][['artist_name', 'artist_popularity']]

,artist_name,artist_popularity
42872,Ariana Grande,100
53392,Drake,98
49775,Post Malone,96
55822,XXXTENTACION,95
49695,Juice WRLD,95
53406,Khalid,95
52306,Ozuna,95
55502,Bad Bunny,94
49320,Travis Scott,94
39126,Queen,94
